In [1]:
from re import compile
from glob import iglob
from pandas import concat, DataFrame, read_csv
from openiti.helper.ara import denormalize

from eis1600.gazetteers.Spellings import Spellings
from eis1600.gazetteers.Toponyms import Toponyms
from eis1600.helper.markdown_patterns import WORD, NOISE_ELEMENTS
from eis1600.helper.repo import MIU_REPO, TOPO_REPO
from eis1600.processing.preprocessing import get_yml_and_miu_df, get_tokens_and_tags
from eis1600.processing.postprocessing import merge_tagslists, reconstruct_miu_text_with_tags

In [2]:
sheets = iglob('../../' + TOPO_REPO + 'sheet_[0-9].csv')
sheets_df = DataFrame(None, columns=['MIU', 'ORIGINAL', 'MODIFIABLE', 'STATUS'])

for sheet in sheets:
    tmp = read_csv(sheet)
    sheets_df = concat([sheets_df, tmp])

entries = sheets_df.loc[sheets_df['STATUS'].isna()]

In [3]:
df_by_files = DataFrame(entries.groupby('MIU')['MODIFIABLE'].apply(list)).reset_index()
infiles = []

for miu in df_by_files['MIU'].to_list():
    author, text, version, uid = miu.split('.')
    file_path = '../../' + MIU_REPO + 'data/' + '/'.join([author, '.'.join([author, text])]) + '/MIUs/' + miu + '.EIS1600'
    infiles.append(file_path)

df_by_files['infile'] = infiles

In [4]:
row = df_by_files.iloc[0]
row

MIU           0240KhalifaIbnKhayyat.Tabaqat.Shia003056-ara1....
MODIFIABLE    [هاشم بن عبد مناف استشهد بسفوان من ناحية البصر...
infile        ../../EIS1600_MIUs/data/0240KhalifaIbnKhayyat/...
Name: 0, dtype: object

In [5]:
place_terms = ['كورة', 'كور', 'قرية', 'قرى', 'مدينة', 'مدن', 'ناحية', 'نواح', 'نواحي', 'محلة', 'محال', 'محلات', 'بلد',
               'بلاد', 'ارباع', 'رستاق', 'رساتيق', 'أعمال']  # 'ربع'
technical_terms = ['من', 'بين',
                   'نسبة',
                   'يوم', 'يوما',
                   'مرحلة', 'مرحلتان', 'مرحلتين', 'مراحل',
                   'فرسخ', 'فرسخا', 'فراسخ',
                   'ميل', 'ميلا', 'أميال']
dn_pt = [denormalize(t) for t in place_terms]
dn_tt = [denormalize(t) for t in technical_terms]
dn_spelling = Spellings.instance().get_denormalized_list()
dn_toponyms = Toponyms.instance().total()

PLACES_REGEX = compile(
        r'(?P<context>(?:' + WORD + NOISE_ELEMENTS +
        r'(?P<spelling> [وب]?(?:ال)?(?:' + '|'.join(dn_spelling) + ')(?:ها)?' + NOISE_ELEMENTS + r')*){1,10})'
        r' (?P<place_term>' + '|'.join(dn_pt) + r')(?P<more_context>' + WORD + NOISE_ELEMENTS + r'){1,7}'
)

TT_REGEX = compile(r'|'.join(dn_pt + dn_tt + dn_spelling + dn_toponyms))

In [6]:
infile = row['infile']
outfile = '../../' + TOPO_REPO + 'MIUs/' + row['MIU'] + '.EIS1600'
with open(infile, 'r', encoding='utf-8') as miu_file_object:
    yml_handler, df = get_yml_and_miu_df(miu_file_object)

text = ' '.join(df['TOKENS'].loc[df['TOKENS'].notna()].to_list())
text_updated = text

if PLACES_REGEX.search(text_updated):
        m = PLACES_REGEX.search(text_updated)
        while m:
            start = m.start()
            end = m.end()
            if len(TT_REGEX.findall(m.group(0))) >= 3:
                p = m.group(0).strip()
                tag = 'TPD' + str(len(p.split())) + ' '
                text_updated = text_updated.replace(p, tag + p)
                m = PLACES_REGEX.search(text_updated, end + len(tag))
            else:
                m = PLACES_REGEX.search(text_updated, end)

الزبير بن العوام بن خويلد بن أسد بن عبد العزى يكنى أبا عبد الله أمه صفية TPD17 بنت عبد المطلب بن هاشم بن عبد مناف استشهد بسفوان من ناحية البصرة سنة ست وثلاثين وأخوه


In [7]:
tokens, tags = get_tokens_and_tags(text_updated)

df.loc[df['TOKENS'].notna(), 'TPD_TAGS'] = tags
df['TAGS_LISTS'] = df.apply(lambda x: merge_tagslists(x['TAGS_LISTS'], x['TPD_TAGS']), axis=1)

updated_text = reconstruct_miu_text_with_tags(df[['SECTIONS', 'TOKENS', 'TAGS_LISTS']])
updated_text

'_ء_#=435133874943= $\n\n_ء_=533398029058= ::UNDEFINED:: ~\n_ء_ الزبير بن العوام بن خويلد بن أسد بن عبد العزى\n\n_ء_=416360170249= ::UNDEFINED:: ~\n_ء_ يكنى أبا عبد الله أمه صفية TPD17 بنت عبد المطلب بن هاشم بن عبد مناف استشهد بسفوان من ناحية البصرة سنة ست وثلاثين\n\n_ء_=465567744777= ::UNDEFINED:: ~\n_ء_ وأخوه\n\n'

In [9]:
with open(outfile, 'w', encoding='utf-8') as outfile_h:
    outfile_h.write(str(yml_handler) + updated_text)


../../Topo_Data/MIUs/0240KhalifaIbnKhayyat.Tabaqat.Shia003056-ara1.435133874943.EIS1600
